# Preprocess: Convert Missing Columns

## Notebook Summary
- Repairs early participant CSV logs with missing or misplaced layer configuration columns.
- Normalizes column order and optionally recomputes `TrialNumber` values for training and experiment blocks.
- Loads participant result JSON files and can copy `Layer01` to `Layer07` back into the tracking CSV rows.
- Writes the repaired CSV to `../data/` and provides a quick verification readback.


## Setup and File Selection
Load required libraries, define paths, and select the participant CSV/JSON files to repair.


In [6]:
# Imports and global constants

# CSV files are located in ../data.

import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import math
import numpy as np
import seaborn as sns

from pandas.plotting import parallel_coordinates

timeFormat = "%Y-%m-%dT%H:%M:%S.%fZ"

export = "../export/"
export_img = "../export/img/"

path = "../data/"

name = 'P03_2022-08-02'

data = glob.glob(rf'{path}/{name}.csv')[0]
results = glob.glob(rf'{path}/{name}.json')[0]


## Preprocessing - Clean Up Logging Issues
- Split pilot tasks and main study into separate files.
- Move `Layer0x` columns to the end.
- Compute `TrialNumber`.
- Copy `Layer0x` columns from `result.json`.
- Write repaired results back to disk.


In [7]:
totalLength = 0
resultlength = 0

numExperimentTrials = 7 * 2
numTrainingTrials = 2

# Toggle options for one-time data repair tasks.
# to convert wrong file format
wrongColumns = True

# only for the pre-study log files: missing columns for Tactile output and layer configuration and display of number (are not restored, but filled with empty values)
shortColumns = False
rewriteTrialNumber = False
computeLayers = False
writeResults = True

df = pd.read_csv(data, sep=";")
fn = os.path.basename(data).split(".")[0]

if wrongColumns:
    # Rebuild the expected CSV schema and move Layer0x columns to the end.
    df = df.reindex(columns=['Date', 'ProbandId', 'Block', 'Trial', 'TrialNumber', 'Condition',
    'ResultNumber', 'ResultLayer', 'ProgramState',
    'ProgramSubState', 'PointsLength', 'PosX', 'PosY', 'PosZ', 'TouchId',
    'Time', 'InfoLength', 'Layer', 'InLayerDepth', 'Deadzone',
    'DataNumberDisplayed', 'IsNumberDisplayed', 'TactileOutput',
    'TactileFeedbackActive', 'IsAtBottom', 'CenterReached', 'Frequency',
    'LayerBorderBump', 'EnterCenterBump', 'inTargetArea', 'delayCount',
    'delayIdx', 'visualizationLastLayerIdx', 'delayElapsedSuccessful', 'Layer01', 'Layer02', 'Layer03',
    'Layer04', 'Layer05', 'Layer06', 'Layer07'])
elif shortColumns:
    # Expand short pre-study logs with missing columns and default placeholders.
    df.columns = ['Date', 'ProbandId', 'Block', 'Trial', 'Condition',
    'ResultNumber', 'ResultLayer', 'ProgramState',
    'ProgramSubState', 'PointsLength', 'PosX', 'PosY', 'PosZ', 'TouchId',
    'Time', 'InfoLength', 'Layer', 'InLayerDepth', 'Deadzone',
    'DataNumberDisplayed', 'IsNumberDisplayed']

    df['IsNumberDisplayed'] = np.where(df['DataNumberDisplayed'] >= 0, True, False)

    df[['TactileOutput', 'TactileFeedbackActive', 'IsAtBottom', 'CenterReached', 'Frequency',
    'LayerBorderBump', 'EnterCenterBump', 'inTargetArea', 'delayCount',
    'delayIdx', 'visualizationLastLayerIdx', 'delayElapsedSuccessful']] = '-'

    df[['Layer01', 'Layer02', 'Layer03', 'Layer04', 'Layer05', 'Layer06', 'Layer07']] = 0

# Ensure numeric typing before trial index computations.
df['Block'] = pd.to_numeric(df['Block'])
df['Trial'] = pd.to_numeric(df['Trial'])

if rewriteTrialNumber:
    # Recompute TrialNumber for training and experiment blocks.
    df['TrialNumber'] = np.where(df['Block'] >= 0, df['Block'] * numExperimentTrials + df['Trial'], df['Block'] * numTrainingTrials + df['Trial'])
df['file'] = fn
totalLength += len(df)

print('totallines: ' + str(totalLength))

df2 = pd.read_json(results, lines=True)
df2.pop('level')
fn = os.path.basename(results).split('.')[0]
df2['file'] = fn

resultlength += len(df2)

# Build the same trial numbering for result rows to align both datasets.
df2['TrialNumber'] = np.where(df2['BlockId'] >= 0, df2['BlockId'] * 21 + df2['TrialId'], df2['BlockId'] * 2 + df2['TrialId'])

if computeLayers:
    # Split layer configuration arrays into explicit Layer01-Layer07 columns.
    df2['Layer01'] = df2['LayerNumberConfiguration'].apply(lambda x: x[0])
    df2['Layer02'] = df2['LayerNumberConfiguration'].apply(lambda x: x[1])
    df2['Layer03'] = df2['LayerNumberConfiguration'].apply(lambda x: x[2])
    df2['Layer04'] = df2['LayerNumberConfiguration'].apply(lambda x: x[3])
    df2['Layer05'] = df2['LayerNumberConfiguration'].apply(lambda x: x[4])
    df2['Layer06'] = df2['LayerNumberConfiguration'].apply(lambda x: x[5])
    df2['Layer07'] = df2['LayerNumberConfiguration'].apply(lambda x: x[6])

    layers = df2[['Layer01', 'Layer02', 'Layer03', 'Layer04', 'Layer05', 'Layer06', 'Layer07', 'TrialNumber']]

    # Update CSV rows by matching TrialNumber between tracking log and results file.
    df.set_index('TrialNumber', inplace=True)
    df.update(layers.set_index('TrialNumber'))
    df = df.reset_index()  # restore the original row structure

    display(df.index)

    df = df.astype({'Layer01': int, 'Layer02': int, 'Layer03': int, 'Layer04': int, 'Layer05': int, 'Layer06': int, 'Layer07': int})

if writeResults:
    # Persist the repaired CSV back into the data directory.
    fn = df.iloc[0]['file']
    display(fn)

    df.pop('file')
    df.to_csv(rf'../data/{fn}.csv', sep=';', index=False)

display(df)
display(df2)

display(df.columns)


totallines: 171345


'P03_2022-08-02'

,Date,ProbandId,Block,Trial,TrialNumber,Condition,ResultNumber,ResultLayer,ProgramState,ProgramSubState,...,delayIdx,visualizationLastLayerIdx,delayElapsedSuccessful,Layer01,Layer02,Layer03,Layer04,Layer05,Layer06,Layer07
0,2022-08-02T06:59:35.741Z,3,-4,0,-8,Combined Feedback,163,6,3,0,...,89,-1,False,146,114,135,141,152,163,149
1,2022-08-02T06:59:35.782Z,3,-4,0,-8,Combined Feedback,163,6,3,0,...,89,-1,False,146,114,135,141,152,163,149
2,2022-08-02T06:59:35.801Z,3,-4,0,-8,Combined Feedback,163,6,3,0,...,89,-1,False,146,114,135,141,152,163,149
3,2022-08-02T06:59:35.833Z,3,-4,0,-8,Combined Feedback,163,6,3,0,...,89,-1,False,146,114,135,141,152,163,149
4,2022-08-02T06:59:35.861Z,3,-4,0,-8,Combined Feedback,163,6,3,0,...,89,-1,False,146,114,135,141,152,163,149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171340,2022-08-02T08:25:18.150Z,3,3,20,83,Combined Feedback,164,1,4,2,...,1354,-1,False,164,144,142,149,133,137,131
171341,2022-08-02T08:25:18.181Z,3,3,20,83,Combined Feedback,164,1,4,2,...,1354,-1,False,164,144,142,149,133,137,131
171342,2022-08-02T08:25:18.210Z,3,3,20,83,Combined Feedback,164,1,4,2,...,1354,-1,False,164,144,142,149,133,137,131
171343,2022-08-02T08:25:18.243Z,3,3,20,83,Combined Feedback,164,1,4,2,...,1354,-1,False,164,144,142,149,133,137,131


,BlockId,CommitResultDate,Condition,EndInteractionDate,ExpectedResultLayer,ExpectedResultNumber,LayerNumberConfiguration,ProbandId,ProvidedResultLayer,ProvidedResultNumber,StartDate,Training,TrialId,file,TrialNumber
0,-4,2022-08-02T07:00:56.085Z,Combined Feedback,2022-08-02T07:00:36.023Z,6,163,"[146, 114, 135, 141, 152, 163, 149]",3,3,168,2022-08-02T06:59:45.375Z,True,0,P03_2022-08-02,-8
1,-4,2022-08-02T07:03:02.564Z,Combined Feedback,2022-08-02T07:02:55.487Z,6,154,"[146, 151, 143, 130, 145, 154, 131]",3,6,154,2022-08-02T07:00:56.086Z,True,1,P03_2022-08-02,-7
2,-3,2022-08-02T07:04:44.293Z,Visual Feedback,2022-08-02T07:04:38.014Z,5,157,"[153, 148, 126, 154, 157, 130, 133]",3,6,157,2022-08-02T07:03:19.727Z,True,0,P03_2022-08-02,-6
3,-3,2022-08-02T07:05:46.941Z,Visual Feedback,2022-08-02T07:05:41.677Z,1,160,"[160, 158, 144, 128, 138, 115, 157]",3,2,160,2022-08-02T07:04:44.295Z,True,1,P03_2022-08-02,-5
4,-2,2022-08-02T07:07:18.909Z,Tactile Feedback,2022-08-02T07:07:13.949Z,7,153,"[152, 146, 149, 119, 134, 147, 153]",3,7,153,2022-08-02T07:06:33.552Z,True,0,P03_2022-08-02,-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,3,2022-08-02T08:22:48.865Z,Combined Feedback,2022-08-02T08:22:45.946Z,4,156,"[140, 155, 141, 156, 123, 138, 148]",3,4,156,2022-08-02T08:22:11.065Z,False,16,P03_2022-08-02,79
88,3,2022-08-02T08:23:28.857Z,Combined Feedback,2022-08-02T08:23:25.449Z,3,154,"[149, 147, 154, 136, 139, 137, 138]",3,3,154,2022-08-02T08:22:48.865Z,False,17,P03_2022-08-02,80
89,3,2022-08-02T08:24:04.954Z,Combined Feedback,2022-08-02T08:24:01.323Z,1,162,"[162, 132, 150, 131, 137, 142, 146]",3,1,162,2022-08-02T08:23:28.858Z,False,18,P03_2022-08-02,81
90,3,2022-08-02T08:24:40.721Z,Combined Feedback,2022-08-02T08:24:37.585Z,6,167,"[136, 127, 149, 142, 147, 167, 132]",3,6,167,2022-08-02T08:24:04.954Z,False,19,P03_2022-08-02,82


Index(['Date', 'ProbandId', 'Block', 'Trial', 'TrialNumber', 'Condition',
       'ResultNumber', 'ResultLayer', 'ProgramState', 'ProgramSubState',
       'PointsLength', 'PosX', 'PosY', 'PosZ', 'TouchId', 'Time', 'InfoLength',
       'Layer', 'InLayerDepth', 'Deadzone', 'DataNumberDisplayed',
       'IsNumberDisplayed', 'TactileOutput', 'TactileFeedbackActive',
       'IsAtBottom', 'CenterReached', 'Frequency', 'LayerBorderBump',
       'EnterCenterBump', 'inTargetArea', 'delayCount', 'delayIdx',
       'visualizationLastLayerIdx', 'delayElapsedSuccessful', 'Layer01',
       'Layer02', 'Layer03', 'Layer04', 'Layer05', 'Layer06', 'Layer07'],
      dtype='object')

## Validate Repaired Column Order
Reload the updated CSV and inspect it to confirm the repair result.


In [63]:
# Reload the output file to verify the repaired column order and values.
df_repaired = pd.read_csv(data, sep=';')

display(df_repaired)


,Date,ProbandId,Block,Trial,Condition,ResultNumber,ResultLayer,ProgramState,ProgramSubState,PointsLength,...,visualizationLastLayerIdx,delayElapsedSuccessful,Layer01,Layer02,Layer03,Layer04,Layer05,Layer06,Layer07,TrialNumber
0,2022-07-29T11:09:06.639Z,0,-4,0,Combined Feedback,162,1,3,0,0,...,-,-,0,0,0,0,0,0,0,-8
1,2022-07-29T11:09:06.748Z,0,-4,0,Combined Feedback,162,1,3,0,0,...,-,-,0,0,0,0,0,0,0,-8
2,2022-07-29T11:09:06.841Z,0,-4,0,Combined Feedback,162,1,3,0,0,...,-,-,0,0,0,0,0,0,0,-8
3,2022-07-29T11:09:06.950Z,0,-4,0,Combined Feedback,162,1,3,0,0,...,-,-,0,0,0,0,0,0,0,-8
4,2022-07-29T11:09:07.044Z,0,-4,0,Combined Feedback,162,1,3,0,0,...,-,-,0,0,0,0,0,0,0,-8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43311,2022-07-29T12:21:17.754Z,0,3,13,Tactile Feedback,149,2,4,2,0,...,-,-,0,0,0,0,0,0,0,55
43312,2022-07-29T12:21:17.848Z,0,3,13,Tactile Feedback,149,2,4,2,0,...,-,-,0,0,0,0,0,0,0,55
43313,2022-07-29T12:21:17.958Z,0,3,13,Tactile Feedback,149,2,4,2,0,...,-,-,0,0,0,0,0,0,0,55
43314,2022-07-29T12:21:18.051Z,0,3,13,Tactile Feedback,149,2,4,2,0,...,-,-,0,0,0,0,0,0,0,55
